# Formulas used in Fractalaxy

 [Fractalaxy](https://github.com/bfxdev/fractalaxy) relies on vector and complex number transformations. This Jupyter notebook shows the underlying concepts and explains where the formulas come from.

## Map Controller 2D

`MapController2D` is GDScript attached to the main rendering node. It receives `InputEvent` signals in its `_input` function and maintain values of exported variables used by the renderer. This script is generic and can be used for any 2D object that can be panned, zoomed or rotated.

The following variables are used for computations and represent the **orthogonal basis of the reference frame in the Map** (typed as `Vector2`):

 - $\vec O$ or `Origin`: coordinates in Map of the point on the top left of the rendered area
 - $\vec X$ or `HorizontalUnit`: distance in Map between two pixels in the horizontal direction to the right of the rendered area
 - $\vec Y$ or `VerticalUnit`: distance in Map between two pixels in the vertical direction to the bottom of the rendered area

The following ones are maintained as alternatives for rendering:

- `Center`: coordinates in Map of the center of the rendered area (`Vector2`)
- `Radius`: the radius of the zone in the rendered area guaranteed to be visible after any resize or switch landscape/portrait